## <center>Error for </center>

\begin{cases}
y'(t)= ay(t-\tau),&\\
y(t)=1 for all t<0&
\end{cases}

## <center>with RK4 </center>

(Take a=\pi / 2)

In [10]:
from math import factorial
import numpy as np

## Right-hand-side of the simple DDE
def simple(a):
    def simple_dde(t, y1, y2):
        return -a*y2
    return simple_dde


## Constant one history function
def phi_0(t):
    return 1


def simple_exact(a, t_0, t_f, tau, N): 
    if t_0 != 0:
        raise ValueError("t_0 should be set to 0")
    t = np.linspace(t_0, t_f, N+1)
    y = np.zeros(len(t))
    for i, ti in enumerate(t):
        n = int(np.floor(ti / tau)) + 1 
        y[i] = 1
        for k in range(1, n+1):
            y[i] += (-a)**k * (ti - (k-1)*tau)**k / factorial(k)
    return y

In [11]:
import numpy as np

## Linear interpolation for delay terms
def y_delayed(t, y, t_q):
    j = np.searchsorted(t, t_q)  #Busca el índice donde insertar t_q para mantener el arreglo ordenado.
    j1, j2 = j-1, j     #Toma los dos puntos de la malla que rodean a t_q
    w = (t_q - t[j1]) / (t[j2] - t[j1])      #Aplica interpolación en estas 2 líneas
    y_delayed = (1 - w) * y[j1] + w * y[j2]
    return y_delayed


def RK4(f, phi, t_0, t_f, tau, N, history=False):

    ## Preliminaries
    h = (t_f - t_0) / N
    t = np.linspace(t_0, t_f, N+1)
    y = np.zeros(len(t))
    y[0] = phi(t_0)

    ## RK4 with linear interpolation for delay term
    for i in range(0, N):
        if t[i] < tau:
            k1 = f(t[i], y[i], phi(t[i] - tau))
            k2 = f(t[i] + 0.5*h, y[i] + h*0.5*k1, phi(t[i] + 0.5*h - tau))
            k3 = f(t[i] + 0.5*h, y[i] + h*0.5*k2, phi(t[i] + 0.5*h - tau))
            k4 = f(t[i] + h, y[i] + h*k3, phi(t[i] + h - tau))
            y[i+1] = y[i] + h*(k1/6 + k2/3 + k3/3 + k4/6)
        else:
            k1 = f(t[i], y[i], y_delayed(t, y, t[i] - tau))
            k2 = f(t[i] + 0.5*h, y[i] + h*0.5*k1, y_delayed(t, y, t[i] + 0.5*h - tau))
            k3 = f(t[i] + 0.5*h, y[i] + h*0.5*k2, y_delayed(t, y, t[i] + 0.5*h - tau))
            k4 = f(t[i] + h, y[i] + h*k3, y_delayed(t, y, t[i] + h - tau))
            y[i+1] = y[i] + h*(k1/6 + k2/3 + k3/3 + k4/6)

    if history == True: ## Wether the solution should include the history (values on the interval [t_0 - tau, t_0])
        t_hist = np.linspace(t_0-tau, t_0, int(N / (t_f // tau) + 1))
        y_hist = np.array([phi(ti) for ti in t_hist])
        t = np.concatenate((t_hist, t))
        y = np.concatenate((y_hist, y))

    return h, t, y

Error with max norm:

In [21]:
#Initial values
a = np.pi / 2
t_0 = 0
t_f = 30
tau = 1


maxnormresult = []
orderresult = []

N = 45 #I changed N (again)

for i in range(0,8):
    h, t, y = RK4(simple(a), phi_0, t_0, t_f, tau, N*2**i, False)
    exact = simple_exact(a, t_0, t_f, tau, N*2**i)
    maxnorm = np.linalg.norm(exact - y,np.inf)
    maxnormresult.append(maxnorm)
    
errorvector = np.hstack(maxnormresult)  


for i in range(0,len(errorvector)-1):
    order = np.log2(errorvector[i]/errorvector[i+1])
    orderresult.append(order)
    
print(orderresult)
print(errorvector)

[np.float64(0.49499096910860924), np.float64(1.7732999490973897), np.float64(1.9423222229598536), np.float64(1.9830161096597096), np.float64(1.9963858419938354), np.float64(1.998851833035911), np.float64(1.9972435257140022)]
[6.60308070e-01 4.68532235e-01 1.37063873e-01 3.56636468e-02
 9.02149305e-03 2.26103038e-03 5.65707633e-04 1.41697383e-04]


RK4 seems perfect, with almost 0 error

Error with one norm:

In [23]:
#Initial values
a = np.pi / 2
t_0 = 0
t_f = 30
tau = 1


onenormresult = []
orderresultone = []

N = 45 #I changed N (again)

for i in range(0,8):
    h, t, y = RK4(simple(a), phi_0, t_0, t_f, tau, N*2**i, False)
    exact = simple_exact(a, t_0, t_f, tau, N*2**i)
    onenorm = h*np.linalg.norm(exact - y,1)   #CREO, NO ESTOY SEGURA
    onenormresult.append(onenorm)
    
errorvectorone = np.hstack(onenormresult)  


for i in range(0,len(errorvectorone)-1):
    orderone = np.log2(errorvectorone[i]/errorvectorone[i+1])
    orderresultone.append(orderone)
    
print(orderresultone)
print(errorvectorone)

[np.float64(0.5617360599082736), np.float64(1.8326554530078032), np.float64(1.9659967378991297), np.float64(1.9910756579562743), np.float64(1.9982573136418438), np.float64(1.9997102981078887), np.float64(1.9999978387460648)]
[7.01159343e+00 4.75025935e+00 1.33362294e+00 3.41357201e-01
 8.58688354e-02 2.14931556e-02 5.37436799e-03 1.34359401e-03]


Error with two norm:

In [24]:
#Initial values
a = np.pi / 2
t_0 = 0
t_f = 30
tau = 1

twonormresult = []
orderresulttwo = []

N = 45 #I changed N (again)

for i in range(0,8):
    h, t, y = RK4(simple(a), phi_0, t_0, t_f, tau, N*2**i, False)
    exact = simple_exact(a, t_0, t_f, tau, N*2**i)
    twonorm = np.sqrt(h) * np.linalg.norm(exact - y)
    twonormresult.append(twonorm)
    
errorvectortwo = np.hstack(twonormresult)  


for i in range(0,len(errorvectortwo)-1):
    ordertwo = np.log2(errorvectortwo[i]/errorvectortwo[i+1])
    orderresulttwo.append(ordertwo)
    
print(orderresulttwo)
print(errorvectortwo)



[np.float64(0.4830670015722332), np.float64(1.8301230006736535), np.float64(1.9565561680705266), np.float64(1.9892864807926502), np.float64(1.9974368785206487), np.float64(1.9994134202738711), np.float64(1.9998616302371741)]
[1.57164059e+00 1.12443817e+00 3.16237713e-01 8.14763501e-02
 2.05209123e-02 5.13935065e-03 1.28536017e-03 3.21370863e-04]
